
## Running through simple computational aids to handwritten sol.

Heuristic: Run through the most constraining expressions ( have to reduce to positive integers) and corroborate validity with other expressions. After, figure out the mosaic component.


In [4]:
import numpy as np  # ty:ignore[unresolved-import]
import sympy as sp  # ty:ignore[unresolved-import]


In [22]:

from sympy.ntheory import primefactors  # ty:ignore[unresolved-import]
import pprint
import polars as pl
import numpy as np

x = np.linspace(-100, 100, 201)
print(x)



# 43 - ac = x ** 3

ac = {int(x): int(43 - x**3) for x in x}



# pprint.pprint(ac)
factors = {a : primefactors(int(e)) for a,e in ac.items()}
# pprint.pprint(factors)

def isvalid(facts : list, x : int) -> bool:
    for f in facts:
        if x % f != 0:
            return False
    return True
valid = {x : facts for x,facts in factors.items() if isvalid(facts, x)}



df = pl.DataFrame({
    "x": list(ac.keys()),
    "x^3": list(ac.values()),
    "ac": [e for e in ac.values()],
    "prime_factors": [str(f) for f in factors.values()],
    "valid": [isvalid(facts, x) for x,facts in factors.items()],
})

with pl.Config(tbl_rows=-1):
    print(df)



[-100.  -99.  -98.  -97.  -96.  -95.  -94.  -93.  -92.  -91.  -90.  -89.
  -88.  -87.  -86.  -85.  -84.  -83.  -82.  -81.  -80.  -79.  -78.  -77.
  -76.  -75.  -74.  -73.  -72.  -71.  -70.  -69.  -68.  -67.  -66.  -65.
  -64.  -63.  -62.  -61.  -60.  -59.  -58.  -57.  -56.  -55.  -54.  -53.
  -52.  -51.  -50.  -49.  -48.  -47.  -46.  -45.  -44.  -43.  -42.  -41.
  -40.  -39.  -38.  -37.  -36.  -35.  -34.  -33.  -32.  -31.  -30.  -29.
  -28.  -27.  -26.  -25.  -24.  -23.  -22.  -21.  -20.  -19.  -18.  -17.
  -16.  -15.  -14.  -13.  -12.  -11.  -10.   -9.   -8.   -7.   -6.   -5.
   -4.   -3.   -2.   -1.    0.    1.    2.    3.    4.    5.    6.    7.
    8.    9.   10.   11.   12.   13.   14.   15.   16.   17.   18.   19.
   20.   21.   22.   23.   24.   25.   26.   27.   28.   29.   30.   31.
   32.   33.   34.   35.   36.   37.   38.   39.   40.   41.   42.   43.
   44.   45.   46.   47.   48.   49.   50.   51.   52.   53.   54.   55.
   56.   57.   58.   59.   60.   61.   62.   63.   

In [24]:
from sympy.ntheory import primefactors
from sympy import divisors
import polars as pl
import numpy as np

x_vals = np.arange(-200, 201).astype(int)

rows = []
for x in x_vals:
    ac = int(43 - x**3)
    if ac == 0:
        rows.append((int(x), x**3, ac, "[]", False, ""))
        continue
    divs = divisors(abs(ac))
    # find any divisor 'a' of ac such that x/a is an integer in [1,16]
    valid_pairs = []
    for a in divs:
        for sign in [1, -1]:
            sa = sign * a
            if sa != 0 and x % sa == 0:
                result = x // sa
                if 1 <= result <= 16:
                    c = ac // sa
                    valid_pairs.append(f"a={sa}, c={c}, x/a={result}")
    pf = primefactors(abs(ac))
    rows.append((int(x), int(x**3), ac, str(pf), len(valid_pairs) > 0, "; ".join(valid_pairs)))

df = pl.DataFrame({
    "x": [r[0] for r in rows],
    "x^3": [r[1] for r in rows],
    "ac": [r[2] for r in rows],
    "prime_factors": [r[3] for r in rows],
    "valid": [r[4] for r in rows],
    "valid_pairs": [r[5] for r in rows],
})

with pl.Config(tbl_rows=-1, tbl_width_chars=200):
    print(df.filter(pl.col("valid")))

shape: (40, 6)
┌──────┬──────────┬──────────┬───────────────────┬───────┬─────────────────────────┐
│ x    ┆ x^3      ┆ ac       ┆ prime_factors     ┆ valid ┆ valid_pairs             │
│ ---  ┆ ---      ┆ ---      ┆ ---               ┆ ---   ┆ ---                     │
│ i64  ┆ i64      ┆ i64      ┆ str               ┆ bool  ┆ str                     │
╞══════╪══════════╪══════════╪═══════════════════╪═══════╪═════════════════════════╡
│ -172 ┆ -5088448 ┆ 5088491  ┆ [17, 43, 6961]    ┆ true  ┆ a=-43, c=-118337, x/a=4 │
│ -129 ┆ -2146689 ┆ 2146732  ┆ [2, 7, 43, 1783]  ┆ true  ┆ a=-43, c=-49924, x/a=3  │
│ -86  ┆ -636056  ┆ 636099   ┆ [3, 43, 4931]     ┆ true  ┆ a=-43, c=-14793, x/a=2  │
│ -43  ┆ -79507   ┆ 79550    ┆ [2, 5, 37, 43]    ┆ true  ┆ a=-43, c=-1850, x/a=1   │
│ -16  ┆ -4096    ┆ 4139     ┆ [4139]            ┆ true  ┆ a=-1, c=-4139, x/a=16   │
│ -15  ┆ -3375    ┆ 3418     ┆ [2, 1709]         ┆ true  ┆ a=-1, c=-3418, x/a=15   │
│ -14  ┆ -2744    ┆ 2787     ┆ [3, 929]          ┆